In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
df = pd.read_csv('/kaggle/input/sample/sample.csv')
df.head()

In [ ]:
df.columns

In [ ]:
prices = df[['AWARD_DATE', 'UNIT_PRICE']]
quantities = df[['AWARD_DATE', 'QUANTITY']]

In [ ]:
prices['DATE'] = pd.to_datetime(prices['AWARD_DATE'])
quantities['DATE'] = pd.to_datetime(quantities['AWARD_DATE'])

In [ ]:
prices.drop(['AWARD_DATE'], inplace = True, axis = 1)

In [ ]:
quantities.drop(['AWARD_DATE'], inplace = True, axis = 1)

In [ ]:
quantities.info()

In [ ]:
prices.describe()

In [ ]:
prices.head()

In [ ]:
spends = quantities['QUANTITY']*prices['UNIT_PRICE']

In [ ]:
spends = spends.to_frame()

In [ ]:
spends['DATE'] = quantities['DATE']

In [ ]:
spends.head()

In [ ]:
spends = spends.rename(columns = {0: "SPENDS"})

In [ ]:
spends = spends[['DATE', 'SPENDS']]

In [ ]:
spends['DATE'].dt.year

In [ ]:
spends = spends.groupby(by = [spends['DATE'].dt.year, spends['DATE'].dt.month]).sum()

In [ ]:
spends

In [ ]:
spends.reset_index(level = 1, inplace = True)

In [ ]:
spends['MONTH'] = spends['DATE']

In [ ]:
spends.drop(columns = ['DATE'], inplace = True, axis = 1)

In [ ]:
spends.reset_index(inplace = True)

In [ ]:
spends.info()

In [ ]:
spends['Y-M'] = spends['DATE'].astype(int).astype(str) + spends['MONTH'].astype(int).astype(str)

In [ ]:
spends.head()

In [ ]:
spends.drop(columns = ['DATE', 'MONTH'], inplace = True, axis = 1)

In [ ]:
spends['DATE'] = spends['Y-M']
spends = spends[['DATE', 'SPENDS']]

In [ ]:
spends.info()

In [ ]:
spends['DATE'] = pd.to_datetime(spends['DATE'], format='%Y%m')

In [ ]:
spends.head()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc

In [ ]:
spends.set_index(pd.DatetimeIndex(spends['DATE']), inplace = True)

In [ ]:
spends = spends.drop(['DATE'], axis = 1)

In [ ]:
spends = spends.sort_index()

In [ ]:
spends = spends.loc[spends.index.notnull()]

In [ ]:
spends.info()

In [ ]:
spends.head()

In [ ]:
plt.rc('figure',figsize=(12,8))
plt.rc('font',size=15)
result = seasonal_decompose(spends, model = 'additive')
fig = result.plot()

In [ ]:
x = result.resid.index
y = result.resid.values
plt.plot_date(x, y, color='black',linestyle='--')
plt.show()

In [ ]:
resid_df = pd.DataFrame(result.resid)

In [ ]:
smooth_path = resid_df.rolling(10).mean()
path_deviation = resid_df.rolling(10).std()
plt.plot(smooth_path, linewidth=2)
plt.plot_date(x, y, color='black')
plt.fill_between(path_deviation.index, (smooth_path-2*path_deviation)['resid'], (smooth_path+2*path_deviation)['resid'], color='b', alpha=.1)

In [ ]:
quantities = quantities[['DATE', 'QUANTITY']]

In [ ]:
quantities = quantities.groupby(by = [quantities['DATE'].dt.year, quantities['DATE'].dt.month]).sum()

In [ ]:
quantities

In [ ]:
quantities.reset_index(level = 1, inplace = True)

In [ ]:
quantities['MONTH'] = quantities['DATE']
quantities.drop(columns = ['DATE'], inplace = True, axis = 1)
quantities.reset_index(inplace = True)
quantities['Y-M'] = quantities['DATE'].astype(int).astype(str) + quantities['MONTH'].astype(int).astype(str)
quantities.drop(columns = ['DATE', 'MONTH'], inplace = True, axis = 1)
quantities['DATE'] = quantities['Y-M']
quantities = quantities[['DATE', 'QUANTITY']]
quantities['DATE'] = pd.to_datetime(quantities['DATE'], format='%Y%m')

In [ ]:
quantities.set_index(pd.DatetimeIndex(quantities['DATE']), inplace = True)
quantities = quantities.drop(['DATE'], axis = 1)
quantities = quantities.sort_index()
quantities = quantities.loc[quantities.index.notnull()]
plt.rc('figure',figsize=(12,8))
plt.rc('font',size=15)
qresult = seasonal_decompose(quantities, model = 'additive')
fig = qresult.plot()

In [ ]:
x = qresult.resid.index
y = qresult.resid.values
plt.plot_date(x, y, color='black',linestyle='--')
plt.show()

In [ ]:
qresid_df = pd.DataFrame(qresult.resid)
smooth_path = qresid_df.rolling(10).mean()
path_deviation = qresid_df.rolling(10).std()
plt.plot(smooth_path, linewidth=2)
plt.plot_date(x, y, color='black')
plt.fill_between(path_deviation.index, (smooth_path-2*path_deviation)['resid'], (smooth_path+2*path_deviation)['resid'], color='b', alpha=.1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [ ]:
#spends.drop(['anomaly'], inplace = True, axis = 1)

In [ ]:
outliers_fraction = float(0.1)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(spends.values.reshape(-1,1))
data = pd.DataFrame(np_scaled)
model = IsolationForest(contamination = outliers_fraction)
model.fit(data)

In [ ]:
spends.head()

In [ ]:
spends['anomaly'] = model.predict(data)
fig, ax = plt.subplots(figsize=(10,6))
a = spends.loc[spends['anomaly'] == -1, ['SPENDS']] #anomaly
ax.plot(spends.index, spends['SPENDS'], color='black', label = 'Normal')
ax.scatter(a.index,a['SPENDS'], color='red', label = 'Anomaly')
plt.legend()
plt.show();

In [ ]:
outliers_fraction = float(0.1)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(quantities.values.reshape(-1,1))
data = pd.DataFrame(np_scaled)
model = IsolationForest(contamination = outliers_fraction)
model.fit(data)

In [ ]:
quantities['anomaly'] = model.predict(data)
fig, ax = plt.subplots(figsize=(10,6))
a = quantities.loc[quantities['anomaly'] == -1, ['QUANTITY']] #anomaly
ax.plot(quantities.index, quantities['QUANTITY'], color='black', label = 'Normal')
ax.scatter(a.index,a['QUANTITY'], color='red', label = 'Anomaly')
plt.legend()
plt.show();

In [ ]:
quantities[quantities['anomaly'] == -1]

In [ ]:
spends[spends['anomaly'] == -1]